# BRANCHES AND SERVER
![](Images/img1.jpg)

## Contenido
* [Git Branching](#Git-Branching)
* [Git en el servidor](#Git-en-el-servidor)

## Git Branching

Casi todos los VCS tienen algún tipo de soporte de ramificación (branching). La ramificación significa que se aparta de la línea principal de desarrollo y continúa trabajando sin meterse con esa línea principal. En muchas herramientas VCS, este es un proceso algo costoso, que a menudo requiere que cree una nueva copia de su directorio de código fuente, lo que puede llevar mucho tiempo para proyectos grandes.

Algunas personas se refieren al modelo de ramificación (branching) de Git como su "característica principal", y ciertamente distingue a Git en la comunidad de VCS. ¿Por qué es tan especial? La forma en que Git se ramifica es increíblemente liviana, lo que hace que las operaciones de ramificación sean casi instantáneas y, en general, el cambio entre las ramas es igual de rápido. A diferencia de muchos otros VCS, Git fomenta los flujos de trabajo que se ramifican y fusionan con frecuencia, incluso varias veces al día. Comprender y dominar esta función le brinda una herramienta poderosa y única y puede cambiar por completo la forma en que se desarrolla.

### Ramas (Branches) en pocas palabras

Para comprender realmente la forma en que Git realiza la ramificación, debemos dar un paso atrás y examinar cómo almacena Git sus datos.

Como recordará de ¿Qué es Git? , Git no almacena datos como una serie de cambios o diferencias, sino como una serie de instantáneas (snapshots).

Cuando realiza una confirmación, Git almacena un objeto de confirmación que contiene un puntero a la instantánea (snapshots) del contenido que organizó. Este objeto también contiene el nombre y la dirección de correo electrónico del autor, el mensaje que escribió y los indicadores de la confirmación o confirmaciones que vinieron directamente antes de esta confirmación (su padre o padres): cero padres para la confirmación inicial, un padre para una confirmación normal y varios padres para una confirmación que resulta de una merge de dos o más ramas.

Para visualizar esto, supongamos que tiene un directorio que contiene tres archivos, los organiza todos y los confirma. La puesta en escena de los archivos calcula una suma de comprobación para cada uno (el hash SHA-1 que mencionamos en ¿Qué es Git? ), Almacena esa versión del archivo en el repositorio de Git (Git se refiere a ellos como **blobs**) y agrega esa suma de comprobación a la preparación área (staging area):

```
$ git add README test.rb LICENSE 
$ git commit -m 'Initial commit'
```

Cuando creas la confirmación ejecutando git commit, Git hace una suma de comprobación de cada subdirectorio (en este caso, solo el directorio raíz del proyecto) y los almacena como un objeto de árbol **(tree)** en el repositorio de Git. Luego, Git crea un objeto de confirmación que tiene los metadatos y un puntero al árbol del proyecto raíz para poder volver a crear esa instantánea (snapshots) cuando sea necesario.

Su repositorio de Git ahora contiene cinco objetos: tres **blobs** (cada uno representa el contenido de uno de los tres archivos), un árbol **(tree)** que enumera el contenido del directorio y especifica qué nombres de archivo se almacenan como qué blobs, y uno se confirma con el puntero a ese árbol raíz y todos los metadatos de confirmación.

![](Images/img2.png)

Si realiza algunos cambios y vuelve a confirmar, la siguiente confirmación almacena un puntero a la confirmación que vino inmediatamente antes.

![](Images/img3.png)

Una rama **(branch)** en Git es simplemente un puntero móvil ligero a una de estas confirmaciones. El nombre de la rama predeterminado en Git es ```master```. A medida que comienza a realizar confirmaciones, se le asigna una rama ```master``` que apunta a la última confirmación que realizó. Cada vez que se compromete, el puntero de la rama ```master``` avanza automáticamente.

**Nota**

La rama **"master"** en Git no es una rama especial. Es exactamente como cualquier otra rama. La única razón por la que casi todos los repositorios tienen uno es que el comando ```git init``` lo crea de forma predeterminada y la mayoría de las personas no se molestan en cambiarlo.
  
![](Images/img4.png)

### Crear una nueva branch

¿Qué sucede cuando crea una nueva rama? Bueno, al hacerlo, se crea un nuevo puntero para que puedas moverte. Digamos que desea crear una nueva rama llamada ```testing```. Haz esto con el comando ```git branch```:

```
$ git branch testing
```

Esto crea un nuevo puntero al mismo compromiso en el que está actualmente.

![](Images/img5.png)

¿Cómo sabe Git en qué rama se encuentra actualmente? Mantiene un puntero especial llamado HEAD. Tenga en cuenta que esto es muy diferente al concepto de HEAD de otros VCS a los que puede estar acostumbrado, como Subversion o CVS. En Git, este es un puntero a la **branch** local en la que se encuentra actualmente. En este caso, todavía está encendido master. El comando **git branch** solo creó una nueva rama **(branch)**, no cambió a esa rama.

![](Images/img6.png)

Puede ver esto fácilmente ejecutando un comando ```git log``` simple que le muestre hacia dónde apuntan los punteros de las ramas. Esta opción se llama ```--decorate```.

```
$ git log --oneline --decorate
f30ab (HEAD -> master, testing) Add feature #32 - ability to add new formats to the central interface
34ac2 Fix bug #1328 - stack overflow under certain conditions
98ca9 Initial commit
```

Puede ver las ramas master y testing que están justo al lado de la confirmación f30ab.

### Cambio de branch

Para cambiar a una **rama (branch)** existente, ejecute el comando ```git checkout```. Cambiemos a la nueva rama ```testing```:

```
$ git checkout testing
```

Esto se mueve para HEAD apuntar a la rama testing.

![](Images/img7.png)

¿Cuál es el significado de eso? Bueno, hagamos otro compromiso:

```
$ vim test.rb
$ git commit -a -m 'made a change'
```

![](Images/img8.png)

Esto es interesante, porque ahora su rama ```testing``` ha avanzado, pero su rama ```master``` todavía apunta a la confirmación en la que estaba cuando corrió ```git checkout``` para cambiar de rama. Volvamos a la rama ```master```:

```
$ git checkout master
```

**Nota**

```git log``` no muestra todas las ramas todo el tiempo
Si tuviera que ejecutar``` git log``` ahora mismo, podría preguntarse a dónde fue la rama de ```"testing"``` que acaba de crear, ya que no aparecería en la salida.

La rama no ha desaparecido; Git simplemente no sabe que estás interesado en esa rama y está tratando de mostrarte lo que cree que te interesa. En otras palabras, de forma predeterminada, ```git log``` solo mostrará el historial de confirmaciones debajo de la rama que has verificado. 

Para mostrar el historial de commits para la rama deseada tiene que especificar explícitamente que branch es: ```git log testing```. Para mostrar todas las ramas, agregue ```--all``` a su comando ```git log```.

![](Images/img9.png)

Ese comando hizo dos cosas. Movió el puntero ```HEAD``` hacia atrás para apuntar a la rama ```master```, y revirtió los archivos en su directorio de trabajo a el snapshot que apunta ```master```. Esto también significa que los cambios que realice a partir de este momento diferirán de una versión anterior del proyecto. Básicamente, rebobina el trabajo que ha realizado en su ```branch testing``` para que pueda ir en una dirección diferente.

**Nota**

Cambiar de rama cambia archivos en su directorio de trabajo

Es importante tener en cuenta que cuando cambia de rama en Git, los archivos en su directorio de trabajo cambiarán. Si cambia a una rama anterior, su directorio de trabajo se revertirá para que se vea como la última vez que se comprometió en esa rama. Si Git no puede hacerlo limpiamente, no le permitirá cambiar en absoluto.

Hagamos algunos cambios y volvamos a comprometernos:

```
$ vim test.rb
$ git commit -a -m 'made other changes'
```

Ahora el historial de su proyecto ha divergido (consulte Historial divergente). Creó y cambió a una rama, hizo algunos trabajos en ella y luego volvió a su rama principal e hizo otro trabajo. Ambos cambios están aislados en ramas separadas: puede alternar entre las ramas y fusionarlas cuando esté listo. Y lo hizo con todo lo sencillo de los comandos **branch**, **checkout** y **commit**.

![](Images/img10.png)

También puede ver esto fácilmente con el comando ```git log```. Si lo ejecuta ```git log --oneline --decorate --graph --all```, imprimirá el historial de sus confirmaciones, mostrando dónde están los punteros de su rama y cómo ha divergido su historial.

```
$ git log --oneline --decorate --graph --all
* c2b9e (HEAD, master) Made other changes
| * 87ab2 (testing) Made a change
|/
* f30ab Add feature #32 - ability to add new formats to the central interface
* 34ac2 Fix bug #1328 - stack overflow under certain conditions
* 98ca9 initial commit of my project
```

Debido a que una rama en Git es en realidad un archivo simple que contiene la suma de comprobación SHA-1 de 40 caracteres de la confirmación a la que apunta, las ramas son baratas de crear y destruir. Crear una nueva rama es tan rápido y simple como escribir 41 bytes en un archivo (40 caracteres y una nueva línea).

Esto contrasta claramente con la forma en que se ramifican la mayoría de las herramientas VCS más antiguas, que implica copiar todos los archivos del proyecto en un segundo directorio. Esto puede llevar varios segundos o incluso minutos, dependiendo del tamaño del proyecto, mientras que en Git el proceso siempre es instantáneo. Además, debido a que registramos a los padres cuando nos comprometemos, encontrar una base de merge adecuada para la merge se realiza automáticamente y, en general, es muy fácil de hacer. Estas funciones ayudan a animar a los desarrolladores a crear y utilizar ramas con frecuencia.

Veamos por qué debería hacerlo.

**Nota**

Crear una nueva rama y cambiar a ella al mismo tiempo

Es típico crear una nueva rama y querer cambiar a esa nueva rama al mismo tiempo; esto se puede hacer en una operación con ```git checkout -b <newbranchname>```.

**Nota**
    
Desde la versión 2.23 de Git en adelante, puede usar ```git switch``` en lugar de ```git checkout``` para:

* Cambiar a una rama existente: ```git switch testing-branch```.

* Crear una nueva rama y cambiar a ella: ```git switch -c new-branch```. La bandera ```-c``` representa a crear, también puede utilizar el indicador completo: ```--create```.

* Volver a su sucursal previamente comprobado: ```git switch -```.

### Bifurcación de Git: bifurcación y fusion (Mergin)

Veamos un ejemplo simple de **ramificación y fusión (branching and merging)** con un flujo de trabajo que podría usar en el mundo real. Seguirás estos pasos:

1. Trabaja un poco en un sitio web.

2. Cree una rama para una nueva historia de usuario en la que está trabajando.

3. Trabaja un poco en esa rama.

En esta etapa, recibirá una llamada de que otro problema es crítico y necesita una revisión. Harás lo siguiente:

1. Cambie a su rama de producción.

2. Cree una rama para agregar la revisión.

3. Una vez que se haya probado, combine la rama de revisión y páselo a producción.

Regrese a su historia de usuario original y continúe trabajando.

### Ramificación (branching) básica

Primero, digamos que está trabajando en su proyecto y ya tiene un par de confirmaciones (commits) en la rama ```master```.

![](Images/img11.png)

Ha decidido que trabajará en el ```issue #53``` en cualquier sistema de seguimiento de problemas que utilice su empresa. Para crear una nueva rama y cambiar a ella al mismo tiempo, puede ejecutar el  comando ```git checkout``` con el interruptor ```-b```:

```
$ git checkout -b iss53
Switched to a new branch "iss53"
```

Esto es una abreviatura de:

```
$ git branch iss53
$ git checkout iss53
```

![](Images/img12.png)

Trabajas en tu sitio web y haces algunos compromisos. Al hacerlo, la rama ```iss53``` avanza, porque la ha verificado (es decir, la está apuntando ```HEAD```):

```
$ vim index.html
$ git commit -a -m 'Create new footer [issue 53]'
```

![](Images/img13.png)

Ahora recibe la llamada de que hay un problema con el sitio web y debe solucionarlo de inmediato. Con Git, no tiene que implementar su corrección junto con los cambios ```iss53``` que ha realizado, y no tiene que esforzarse mucho en revertir esos cambios antes de poder trabajar en la aplicación de su corrección a lo que está en producción. Todo lo que tienes que hacer es volver a tu ```branch master```.

Sin embargo, antes de hacer eso, tenga en cuenta que si su directorio de trabajo (working directory) o área de ensayo (staging area) tiene cambios no confirmados que entran en conflicto con la rama que está revisando, Git no le permitirá cambiar de rama. Es mejor tener un estado de funcionamiento limpio cuando cambia de rama. Hay formas de evitar esto (es decir, esconder y cometer enmiendas) que cubriremos más adelante, en Almacenamiento y limpieza. Por ahora, supongamos que ha confirmado todos sus cambios, para que pueda volver a su ```rama master```:

```
$ git checkout master
Switched to branch 'master'
```

En este punto, el directorio de trabajo (working directory) de su proyecto está exactamente como estaba antes de comenzar a trabajar en el **problema (issue) #53**, y puede concentrarse en su revisión. Este es un punto importante para recordar: cuando cambia de rama, Git restablece su directorio de trabajo para que se vea como lo hizo la última vez que se comprometió en esa rama. Agrega, elimina y modifica archivos automáticamente para asegurarse de que su copia de trabajo sea como se veía la rama en su última confirmación.

A continuación, tiene que hacer una revisión. Creemos una rama ```hotfix``` en la que trabajar hasta que esté completa:

```
$ git checkout -b hotfix
Switched to a new branch 'hotfix'
$ vim index.html
$ git commit -a -m 'Fix broken email address'
[hotfix 1fb7853] Fix broken email address
 1 file changed, 2 insertions(+)
```

![](Images/img14.png)
 
Puede ejecutar sus pruebas, asegurarse de que la revisión sea lo que desea y, finalmente, fusionar la rama ```hotfix``` de nuevo en su rama ```master``` para implementarla en producción. Haz esto con el comando ```git merge```:

```
$ git checkout master
$ git merge hotfix
Updating f42c576..3a0874c
Fast-forward
 index.html | 2 ++
 1 file changed, 2 insertions(+)
```

Notarás la frase ```"Fast-forward"``` en ese ```merge```. Debido a que la confirmación ```C4``` señalada por la rama ```hotfix``` en la  que se fusionó estaba directamente antes de la confirmación en la ```C2``` que se encuentra, Git simplemente mueve el puntero hacia adelante. Para expresarlo de otra manera, cuando intentas fusionar una confirmación con una confirmación que se puede alcanzar siguiendo el historial de la primera confirmación, Git simplifica las cosas moviendo el puntero hacia adelante porque no hay trabajo divergente para fusionar; esto se llama un ```"avance rápido."```

Su cambio ahora está en el snapshot de la confirmación señalada por la rama ```master```, y puede implementar la corrección.

![](Images/img15.png)

Después de implementar su solución súper importante, estará listo para volver al trabajo que estaba haciendo antes de que lo interrumpieran. Sin embargo, primero eliminará la rama ```hotfix```, porque ya no la necesita: la rama ```master``` apunta en el mismo lugar. Puedes eliminarlo con la opción ```-d``` de ```git branch```:

```
$ git branch -d hotfix
Deleted branch hotfix (3a0874c).
```

Ahora puede volver a su rama de trabajo en progreso en el iisue #53 y continuar trabajando en él.

```
$ git checkout iss53
Switched to branch "iss53"
$ vim index.html
$ git commit -a -m 'Finish the new footer [issue 53]'
[iss53 ad82d7a] Finish the new footer [issue 53]
1 file changed, 1 insertion(+)
```

![](Images/img16.png)

Vale la pena señalar aquí que el trabajo que hizo en su rama ```hotfix``` no está contenido en los archivos de su rama ```iss53```. Si necesita incorporarlo, puede fusionar su rama master en su rama ```iss53``` ejecutándo ```git merge master```, o puede esperar para integrar esos cambios hasta que decida volver a introducir la rama ```iss53``` a la ```master``` más tarde.

### Fusión (Merge) básica

Suponga que ha decidido que el trabajo de ```issue #53``` está completo y listo para fusionarse en su rama ```master```. Para hacer eso, fusionará su rama ```iss53``` en ```master```, al igual que fusionó su rama ```hotfix``` anteriormente. Todo lo que tiene que hacer es verificar la rama en la que desea fusionarse y luego ejecutar el comando ```git merge```:

```
$ git checkout master
Switched to branch 'master'
$ git merge iss53
Merge made by the 'recursive' strategy.
index.html |    1 +
1 file changed, 1 insertion(+)
```

Esto se ve un poco diferente al merge de ```hotfix``` que hizo anteriormente. En este caso, su historial de desarrollo ha diferido de algún punto anterior. Debido a que la confirmación en la rama en la que se encuentra no es un antepasado directo de la rama en la que se está fusionando, Git tiene que trabajar un poco. En este caso, Git realiza un **merge** simple de tres vías, utilizando los dos snapshots señalados por las puntas de las ramas y el ancestro común de las dos.

![](Images/img17.png)

En lugar de simplemente mover el puntero de la rama hacia adelante, Git crea una nueva snapshot que resulta de esta merge de tres vías y crea automáticamente un nuevo commit que apunta a el. Esto se conoce como un ```merge commit``` y es especial porque tiene más de un padre.

![](Images/img18.png)

Ahora que su trabajo está fusionado, no necesita más la rama ```iss53```. Puede cerrar el problema en su sistema de seguimiento de problemas y eliminar la rama:

```
$ git branch -d iss53
```

### Conflictos básicos de fusión (merge)

De vez en cuando, este proceso no se desarrolla sin problemas. Si cambió la misma parte del mismo archivo de manera diferente en las dos ramas que está fusionando, Git no podrá fusionarlas limpiamente. Si su solución para el ```issue #53``` modificó la misma parte de un archivo que la rama ```hotfix```, obtendrá un conflicto de fusión que se parece a esto:

```
$ git merge iss53
Auto-merging index.html
CONFLICT (content): Merge conflict in index.html
Automatic merge failed; fix conflicts and then commit the result.
```

Git no ha creado automáticamente un nuevo ```merge commit```. Ha detenido el proceso mientras resuelve el conflicto. Si desea ver qué archivos no se fusionaron en algún momento después de un conflicto de merge, puede ejecutar ```git status```:

```
$ git status
On branch master
You have unmerged paths.
  (fix conflicts and run "git commit")

Unmerged paths:
  (use "git add <file>..." to mark resolution)

    both modified:      index.html

no changes added to commit (use "git add" and/or "git commit -a")
```
    
Todo lo que tenga conflictos de fusión y no se haya resuelto aparecerá como no fusionado. Git agrega marcadores de resolución de conflictos estándar a los archivos que tienen conflictos, por lo que puede abrirlos manualmente y resolver esos conflictos. Su archivo contiene una sección que se parece a esto:

```    
<<<<<<< HEAD:index.html
<div id="footer">contact : email.support@github.com</div>
=======
<div id="footer">
 please contact us at support@github.com
</div>
>>>>>>> iss53:index.html
```

Esto significa que la versión en HEAD(su rama ```master```, porque eso era lo que había verificado cuando ejecutó su comando de fusión) es la parte superior de ese bloque (todo lo que está arriba =======), mientras que la versión en su rama ```iss53``` se ve como todo en la parte inferior. Para resolver el conflicto, debe elegir un lado o el otro o fusionar los contenidos usted mismo. Por ejemplo, puede resolver este conflicto reemplazando todo el bloque con esto:

```
<div id="footer">
please contact us at email.support@github.com
</div>
```

Esta resolución tiene un poco de cada sección y el <<<<<<<, =======y >>>>>>>las líneas han sido completamente eliminado. Una vez que haya resuelto cada una de estas secciones en cada archivo en conflicto, ejecute cada ```git add``` a archivo para marcarlo como resuelto. La puesta en escena del archivo lo marca como resuelto en Git.

Si desea utilizar una herramienta gráfica para resolver estos problemas, puede ejecutar ```git mergetool```, lo que activa una herramienta de merge visual adecuada y lo guía a través de los conflictos:

```
$ git mergetool

This message is displayed because 'merge.tool' is not configured.
See 'git mergetool --tool-help' or 'git help config' for more details.
'git mergetool' will now attempt to use one of the following tools:
opendiff kdiff3 tkdiff xxdiff meld tortoisemerge gvimdiff diffuse diffmerge ecmerge p4merge araxis bc3 codecompare vimdiff emerge
Merging:
index.html

Normal merge conflict for 'index.html':
  {local}: modified file
  {remote}: modified file
Hit return to start merge resolution tool (opendiff):
```

Si desea utilizar una herramienta de merge que no sea la predeterminada (Git eligió ```opendiff``` en este caso porque el comando se ejecutó en una Mac), puede ver todas las herramientas compatibles enumeradas en la parte superior después de "una de las siguientes herramientas". Simplemente escriba el nombre de la herramienta que prefiere utilizar.

**Nota**

Si necesita herramientas más avanzadas para resolver conflictos de merge complicados, cubrimos más sobre la merge en merge avanzada .

Después de salir de la herramienta de merge, Git le pregunta si la merge se realizó correctamente. Si le dice a la secuencia de comandos que lo fue, prepara el archivo para marcarlo como resuelto por usted. Puede ejecutar el ```git status``` de nuevo para verificar que se hayan resuelto todos los conflictos:

```
$ git status
On branch master
All conflicts fixed but you are still merging.
  (use "git commit" to conclude merge)

Changes to be committed:

    modified:   index.html
```

Si está satisfecho con eso y verifica que todo lo que tenía conflictos se ha preparado, puede escribir ```git commit``` para finalizar la confirmación de fusión (merge commit). El mensaje de confirmación por defecto se parece a esto:
```
Merge branch 'iss53'

Conflicts:
    index.html
#
# It looks like you may be committing a merge.
# If this is not correct, please remove the file
#	.git/MERGE_HEAD
# and try again.


# Please enter the commit message for your changes. Lines starting
# with '#' will be ignored, and an empty message aborts the commit.
# On branch master
# All conflicts fixed but you are still merging.
#
# Changes to be committed:
#	modified:   index.html
#
```

Si cree que sería útil para otras personas que vean esta fusión en el futuro, puede modificar este mensaje de confirmación con detalles sobre cómo resolvió la fusión **(merge)** y explicar por qué hizo los cambios que realizó si estos no son obvios.

### Administración de Branches

Ahora que ha creado, fusionado (merge) y eliminado algunas ramas (branch), veamos algunas herramientas de administración de ramas que serán útiles cuando comience a usar las ramas todo el tiempo.

El comando ```git branch``` hace más que crear y eliminar ramas. Si lo ejecuta sin argumentos, obtiene una lista simple de sus ramas actuales:

```
$ git branch
  iss53
* master
  testing
```

Observe el carácter ```*``` que antepone la rama ```master```: indica la rama que ha desprotegido actualmente (es decir, la rama a la que apunta ```HEAD```). Esto significa que si se compromete en este punto, la rama ```master``` avanzará con su nuevo trabajo. Para ver la última confirmación en cada rama, puede ejecutar ```git branch -v```:

```
$ git branch -v
  iss53   93b412c Fix javascript issue
* master  7a98805 Merge branch 'iss53'
  testing 782fd34 Add scott to the author list in the readme
```

Las opciones útiles ```--merged``` y ```--no-merged``` pueden filtrar esta lista a las ramas que tiene o que aún no se han fusionado en la rama en la que se encuentra actualmente. Para ver qué ramas ya están fusionadas con la rama en la que se encuentra, puede ejecutar ```git branch --merged```:

```
$ git branch --merged
  iss53
* master
```

Como ya se fusionó ```iss53``` anteriormente, lo verá en su lista. Las ramas de esta lista sin el ```*``` frente de ellas generalmente están bien para eliminarlas con ```git branch -d```; ya has incorporado su trabajo en otra rama, por lo que no vas a perder nada.

Para ver todas las ramas que contienen trabajo en el que aún no se ha fusionado, puede ejecutar ```git branch --no-merged```:

```
$ git branch --no-merged
  testing
```

Esto muestra su otra rama. Debido a que contiene trabajo que aún no se ha fusionado, intentar eliminarlo con ```git branch -d``` fallará:

```
$ git branch -d testing
error: The branch 'testing' is not fully merged.
If you are sure you want to delete it, run 'git branch -D testing'.
```

Si realmente desea eliminar la rama y perder ese trabajo, puede forzarlo con ```-D```, como señala el útil mensaje.

**Consejo**

Las opciones descritas anteriormente, ```--merged``` y ```--no-merged```, si no se le da un compromiso o un nombre de rama como argumento, le mostrarán lo que se fusionó o no, respectivamente, en su rama actual.

Siempre puede proporcionar un argumento adicional para preguntar sobre el estado de fusión con respecto a alguna otra rama sin verificar primero esa otra rama, como en, ¿qué no se fusiona en la rama master?

```
$ git checkout testing
$ git branch --no-merged master
  topicA
  featureB
```

### Cambiar el nombre de una branch

**Precaución**

No cambie el nombre de las ramas que todavía están en uso por otros colaboradores. No cambie el nombre de una rama como ```master / main / mainline``` sin haber leído la sección "Cambiar el nombre de la rama ```master```".

Suponga que tiene una rama que se llama ```bad-branch-name``` y desea cambiarla ```corrected-branch-name```, manteniendo todo el historial. También desea cambiar el nombre de la rama en el remoto ```(GitHub, GitLab, otro servidor)```. ¿Cómo haces esto?

Cambie el nombre de la rama localmente con el comando ```git branch --move```:

```
$ git branch --move bad-branch-name corrected-branch-name
```

Esto reemplaza su ```bad-branch-name``` con ```corrected-branch-name```, pero este cambio es solo local por ahora. Para permitir que otros vean la rama corregida en el control remoto, empújela (push):

```
$ git push --set-upstream origin corrected-branch-name
```

Ahora veremos brevemente dónde estamos ahora:

```
$ git branch --all
* corrected-branch-name
  main
  remotes/origin/bad-branch-name
  remotes/origin/corrected-branch-name
  remotes/origin/main
```

Observe que esta branch ```corrected-branch-name``` y está disponible en el control remoto. Sin embargo, la rama con el nombre incorrecto también está presente allí, pero puede eliminarla ejecutando el siguiente comando:

```
$ git push origin --delete bad-branch-name
```

Ahora el nombre incorrecto de la rama se reemplaza por completo con el nombre de la rama corregido.

### Cambiar el nombre de la rama ```master```

**Advertencia**

Cambiar el nombre de una rama como ```master / main / mainline / default``` romperá las integraciones, servicios, utilidades auxiliares y scripts de compilación / lanzamiento que usa su repositorio. Antes de hacer esto, asegúrese de consultar con sus colaboradores. Además, asegúrese de realizar una búsqueda exhaustiva a través de su repositorio y actualice cualquier referencia al nombre de la rama anterior en su código y scripts.

Cambie el nombre de su ```branch master``` local a ```main``` con  el siguiente comando:

```
$ git branch --move master main
```

Ya no hay una branch ```master``` local , porque se le cambió el nombre a la branch ```main```.

Para permitir que otros vean la nueva branch ```main```, debe empujarla (push) hacia el servidor remoto. Esto hace que la rama renombrada esté disponible en el servidor remoto.

```
$ git push --set-upstream origin main
```

Ahora terminamos con el siguiente estado:

```
git branch --all
* main
  remotes/origin/HEAD -> origin/master
  remotes/origin/main
  remotes/origin/master
```

Su branch ```master``` local desapareció, ya que fue reemplazada por la branch ```main```. La rama ```main``` está presente en el servidor remoto. Sin embargo, la rama ```master``` antigua todavía está presente en el servidor remoto. Otros colaboradores seguirán utilizando la rama ```master``` como base de su trabajo, hasta que realice algunos cambios adicionales.

Ahora tiene algunas tareas más frente a usted para completar la transición:

* Cualquier proyecto que dependa de este deberá actualizar su código y / o configuración.

* Actualice los archivos de configuración del ejecutor de pruebas.

* Ajuste los scripts de creación y lanzamiento.

* Redirija la configuración en el host de su repositorio para cosas como la rama predeterminada del repositorio, las reglas de fusión y otras cosas que coincidan con los nombres de las ramas.

* Actualice las referencias a la rama anterior en la documentación.

* Cierre o combine cualquier solicitud de extracción que tenga como destino la rama anterior.

Una vez que haya realizado todas estas tareas y esté seguro de que la que rama ```main``` funciona igual que la rama ```master```, puede eliminar la rama  ```master```:

```
$ git push origin --delete master
```

### Flujos de trabajo de branch

Ahora que tiene los conceptos básicos de ramificación **(branching) y fusión (merge)**, ¿qué puede o debe hacer con ellos? En esta sección, cubriremos algunos flujos de trabajo comunes que hace posible esta branching ligera, para que pueda decidir si desea incorporarlos en su propio ciclo de desarrollo.

### Ramas de larga duración

Debido a que Git usa una merge simple de tres vías, la merge de una rama a otra varias veces durante un período prolongado es generalmente fácil de hacer. Esto significa que puede tener varias ramas que estén siempre abiertas y que utilice para diferentes etapas de su ciclo de desarrollo; puede fusionar regularmente de algunos de ellos a otros.

Muchos desarrolladores de Git tienen un flujo de trabajo que adopta este enfoque, como tener solo código que sea completamente estable en su rama ``````master``````, posiblemente solo código que ha sido o será lanzado. Tienen otra rama paralela nombrada ``````develop`````` o ``````next`````` desde la que trabajan o usan para probar la estabilidad; no siempre es necesariamente estable, pero siempre que llega a un estado estable, se puede fusionar con ``````master``````. Se usa para extraer ramas temáticas (ramas de corta duración, como la rama ``````iss53`````` anterior) cuando están listas, para asegurarse de que pasen todas las pruebas y no introduzcan errores.

En realidad, estamos hablando de punteros que avanzan en la línea de confirmaciones que estás realizando. Las ramas estables están más abajo en la línea de su historial de confirmaciones, y las ramas más avanzadas están más arriba en la historia.

![](Images/img19.png)

En general, es más fácil pensar en ellos como silos de trabajo, donde los conjuntos de confirmaciones pasan a un silo más estable cuando se prueban por completo.

![](Images/img20.png)

Puede seguir haciendo esto durante varios niveles de estabilidad. Algunos proyectos más grandes también tienen una rama proposed o pu (actualizaciones propuestas) que tiene ramas integradas que pueden no estar listas para entrar en la rama ``````next`````` o ``````master``````. La idea es que sus ramas se encuentren en varios niveles de estabilidad; cuando alcanzan un nivel más estable, se fusionan con la rama sobre ellos. Una vez más, no es necesario tener varias ramas de larga duración, pero a menudo es útil, especialmente cuando se trata de proyectos muy grandes o complejos.

### Ramas temáticas (topic)

Las ramas temáticas (topic), sin embargo, son útiles en proyectos de cualquier tamaño. Una rama de tema (branch topic) es una rama de corta duración que se crea y usa para una sola función en particular o trabajo relacionado. Esto es algo que probablemente nunca haya hecho antes con un VCS porque generalmente es demasiado costoso crear y fusionar ramas. Pero en Git es común crear, trabajar, fusionar y eliminar ramas varias veces al día.

Viste esto en la última sección con las ramas ```iss53``` y ```hotfix``` que creaste. Hizo algunas confirmaciones en ellos y los eliminó directamente después de fusionarlos en su rama principal. Esta técnica le permite cambiar de contexto de manera rápida y completa, ya que su trabajo está separado en silos donde todos los cambios en esa rama tienen que ver con ese tema, es más fácil ver lo que sucedió durante la revisión del código y demás. Puede mantener los cambios allí durante minutos, días o meses, y fusionarlos cuando estén listos, independientemente del orden en el que se crearon o en el que se trabajaron.

Considere un ejemplo de hacer algo de trabajo (en ```master```), bifurcarse por un problema (```iss91```), trabajar un poco en él, bifurcarse en la segunda bifurcación para probar otra forma de manejar lo mismo (```iss91v2```), volver a su bifurcación ```master``` y trabajar allí por un tiempo, y luego bifurcarse allí para hacer un trabajo que no está seguro de que sea una buena idea (bifurcación ```dumbidea```). Su historial de confirmaciones se verá así:

![](Images/img21.png)

Ahora, digamos que decide que le gusta más la segunda solución a su problema (```iss91v2```); y les mostró la rama  ```dumbidea``` a sus compañeros de trabajo, y resultó ser genial. Puede deshacerse de la rama ```iss91``` original (perder confirmaciones ```C5``` y ```C6```) y fusionar las otras dos. Su historial entonces se ve así:

![](Images/img22.png)
Figura. Historial después de fusionar ```dumbidea``` y ```iss91v2```
 
Entraremos en más detalles sobre los diversos flujos de trabajo posibles para su proyecto Git en Distributed Git , por lo que antes de decidir qué esquema de ramificación utilizará su próximo proyecto, asegúrese de leer ese capítulo.

Es importante recordar al hacer todo esto que estas branching son completamente locales. Cuando está ramificando y fusionando, todo se hace solo en su repositorio de Git, no hay comunicación con el servidor.

### Ramas Remotas

Las referencias remotas son referencias (punteros) en sus repositorios remotos, incluidas ramas, etiquetas, etc. Puede obtener una lista completa de referencias remotas explícitamente con ```git ls-remote <remote>```, o ```git remote show <remote>``` para branches remotas, así como más información. Sin embargo, una forma más común es aprovechar las branches de seguimiento remoto.

Las ramas de seguimiento remoto son referencias al estado de las ramas remotas. Son referencias locales que no puedes mover; Git los mueve por usted cada vez que realiza cualquier comunicación de red, para asegurarse de que representen con precisión el estado del repositorio remoto. Piense en ellos como marcadores, para recordarle dónde estaban las branches en sus repositorios remotos la última vez que se conectó a ellas.

Los nombres de las branches de seguimiento remoto toman la forma ```<remote>/<branch>```. Por ejemplo, si quisiera ver cómo era la rama ```master``` en su servidor remoto origin la última vez que se comunicó con ella, debería verificar la rama ```origin/master```. Si estaba trabajando en un problema con un socio y este subió una branch ```iss53```, es posible que tenga su propia branch local ```iss53```, pero la branch en el servidor estaría representada por la branch de seguimiento remoto ```origin/iss53```.

Esto puede resultar un poco confuso, así que veamos un ejemplo. Digamos que tiene un servidor Git en su red en ```git.ourcompany.com```. Si clona a partir de esto, el comando ```clone``` de Git lo nombra automáticamente ```origin``` por usted, extrae todos sus datos, crea un puntero hacia donde está su rama ```master``` y lo nombra ```origin/master``` localmente. Git también te ofrece tu propia branch ```master``` local que comienza en el mismo lugar que la branch ```master``` de ```origin``` , por lo que tienes algo desde lo que trabajar.
 
**Nota**
    
```"Origin"``` no es especial

Al igual que el nombre de la rama ```"master"``` no tiene ningún significado especial en Git, tampoco lo tiene ```"origin"```. Mientras que ```"master"``` es el nombre predeterminado para una rama de inicio cuando se ejecuta ```git init```, lo cual es la única razón por la que se usa ampliamente, ```"origin"``` es el nombre predeterminado para un servidor remoto cuando se ejecuta ```git clone```. Si ejecuta en su lugar ```git clone -o booyah```, tendrá ```booyah/master``` como rama remota predeterminada.
    
![](Images/img23.png)

Si trabaja un poco en su branch ```master``` local y, mientras tanto, alguien más presiona ```git.ourcompany.com``` y actualiza su branch ```master```, entonces sus historiales avanzan de manera diferente. Además, mientras permanezca fuera de contacto con su servidor origin, su puntero ```origin/master``` no se moverá.

![](Images/img24.png)
    
Para sincronizar su trabajo con un servidor remoto dado, ejecuta un comando ```git fetch <remote>``` (en nuestro caso, ```git fetch origin```). Este comando busca qué servidor ```"origin"``` es (en este caso, es ```git.ourcompany.com```), obtiene cualquier dato de él que aún no tiene y actualiza su base de datos local, moviendo su puntero ```origin/master``` a su nuevo, más actualizada posición.

![](Images/img25.png)
    
Para demostrar que tiene varios servidores remotos y cómo se ven las branches remotas para esos proyectos remotos, supongamos que tiene otro servidor Git interno que solo uno de sus equipos de sprint usa para el desarrollo. Este servidor está en ```git.team1.ourcompany.com```. Puede agregarlo como una nueva referencia remota al proyecto en el que está trabajando actualmente ejecutando el comando ```git remote add``` como cubrimos en Conceptos básicos de Git. Nombra este servidor remoto ```teamone```, que será tu nombre corto para toda esa URL.

![](Images/img26.png)
    
Ahora, puede ejecutar ```git fetch teamone``` para buscar todo lo que  teamone tiene en el servidor remoto y que usted todavía no tiene. Debido a que ese servidor tiene un subconjunto de los datos que su servidor ```origin``` tiene en este momento, Git no obtiene datos, pero establece una rama de seguimiento remoto llamada ```teamone/master``` para apuntar a la confirmación que ```teamone``` tiene como rama ```master```.
    
![](Images/img27.png)
    
### Emprendedor (Pushing)
    
Cuando desee compartir una branch con el mundo, debe enviarla a un servidor remoto al que tenga acceso de escritura. Tus branches locales no se sincronizan automáticamente con los servidores remotos a los que escribes; debes presionar explícitamente las branches que deseas compartir. De esa manera, puede usar las ramas privadas para el trabajo que no desea compartir y **empujar (push)** hacia arriba solo las ramas temáticas (topic) en las que desea colaborar.

Si tiene una rama nombrada ```serverfix``` en la que desea trabajar con otras personas, puede subirla de la misma manera que empujó (push) su primera rama. Ejecutar ```git push <remote> <branch>```:

```    
$ git push origin serverfix
Counting objects: 24, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (15/15), done.
Writing objects: 100% (24/24), 1.91 KiB | 0 bytes/s, done.
Total 24 (delta 2), reused 0 (delta 0)
To https://github.com/schacon/simplegit
 * [new branch]      serverfix -> serverfix
```

Este es un atajo. Git expande automáticamente el nombre ```serverfix``` de la branch a ```refs/heads/serverfix:refs/heads/serverfix```, lo que significa, "Toma mi branch ```serverfix``` local y empújala para actualizar la branch ```serverfix``` remota ". Repasaremos la parte ```refs/heads/``` en detalle en Git Internals, pero generalmente puede omitirla. También puede hacer ```git push origin serverfix:serverfix```, que hace lo mismo: dice: "Tome mi ```servidorfix``` y conviértalo en el servidor remoto". Puede utilizar este formato para insertar una branch local en una branch remota con un nombre diferente. Si no desea que se llame ```serverfix``` en el servidor remoto, puede ejecutar ```git push origin serverfix:awesomebranch``` en su lugar para enviar su branch ```serverfix``` local a la branch ```awesomebranch``` en el proyecto remoto.

**Nota**

No escriba su contraseña cada vez

Si está utilizando una URL HTTPS para enviar, el servidor de Git le pedirá su nombre de usuario y contraseña para la autenticación. De forma predeterminada, le solicitará esta información en la terminal para que el servidor pueda saber si tiene permiso para hacer push.

Si no desea escribirlo cada vez que haga push, puede configurar un "caché de credenciales". Lo más simple es simplemente guardarlo en la memoria durante unos minutos, que puede configurar fácilmente ejecutándo ```git config --global credential.helper cache```.

Para obtener más información sobre las diversas opciones de almacenamiento en caché de credenciales disponibles, consulte Almacenamiento de credenciales.

La próxima vez que uno de sus colaboradores haga un ```fetch``` del servidor, obtendrá una referencia de dónde se encuentra la versión del servidor ```serverfix``` en la rama remota ```origin/serverfix```:

```
$ git fetch origin
remote: Counting objects: 7, done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 3 (delta 0)
Unpacking objects: 100% (3/3), done.
From https://github.com/schacon/simplegit
 * [new branch]      serverfix    -> origin/serverfix
```

Es importante tener en cuenta que cuando realiza una búsqueda que derriba nuevas ramas de seguimiento remoto, no tiene automáticamente copias locales editables de ellas. En otras palabras, en este caso, no tiene una nueva rama ```serverfix```, solo tiene un  puntero ```origin/serverfix``` que no puede modificar.

Para fusionar (merge) este trabajo en su rama de trabajo actual, puede ejecutar ```git merge origin/serverfix```. Si desea tener su propia rama ```serverfix``` en la que pueda trabajar, puede basarla en su rama de seguimiento remoto:

```    
$ git checkout -b serverfix origin/serverfix
Branch serverfix set up to track remote branch serverfix from origin.
Switched to a new branch 'serverfix'
```

Esto le brinda una branch local en la que puede trabajar y que comienza donde está ```origin/serverfix```.

### Seguimiento (Tracking) de ramas

Verificar una rama local desde una rama de seguimiento remoto crea automáticamente lo que se llama una "rama de seguimiento" (“tracking branch”) (y la rama que rastrea se llama una "rama ascendente" (“upstream branch”)). Las branches de seguimiento son branches locales que tienen una relación directa con una branch remota. Si está en una rama de seguimiento y escribe ```git pull```, Git sabe automáticamente de qué servidor buscar y en qué rama fusionarse.

Cuando clona un repositorio, generalmente crea automáticamente una rama master que realiza un seguimiento ```origin/master```. Sin embargo, puede configurar otras ramas de seguimiento si lo desea, las que rastrean las ramas en otros servidores remotos o no rastrean la rama ```master```. El caso simple es el ejemplo que acaba de ver, en ejecución ```git checkout -b <branch> <remote>/<branch>```. Esta es una operación bastante común que Git proporciona la abreviatura ```--track```:

```
$ git checkout --track origin/serverfix
Branch serverfix set up to track remote branch serverfix from origin.
Switched to a new branch 'serverfix'
```    
    
De hecho, esto es tan común que incluso hay un atajo para ese atajo. Si el nombre de la rama que está intentando ```checkout``` (a) no existe y (b) coincide exactamente con un nombre en un solo servidor remoto, Git creará una rama de seguimiento para usted:

```    
$ git checkout serverfix
Branch serverfix set up to track remote branch serverfix from origin.
Switched to a new branch 'serverfix'
```

Para configurar una branch local con un nombre diferente al de la branch remota, puede usar fácilmente la primera versión con un nombre de branch local diferente:

```
$ git checkout -b sf origin/serverfix
Branch sf set up to track remote branch serverfix from origin.
Switched to a new branch 'sf'
```    
    
Ahora, su branch local ```sf``` se extraerá automáticamente de ```origin/serverfix```.

Si ya tiene una branch local y desea configurarla en una branch remota que acaba de abrir, o desea cambiar la branch ```upstream``` que está rastreando, puede usar la opción ```-u``` o ```--set-upstream-to``` opcion de ```git branch```  para configurarla explícitamente en cualquier momento.
    
```    
$ git branch -u origin/serverfix
Branch serverfix set up to track remote branch serverfix from origin.
```

**Nota**

Taquigrafía corriente arriba

Cuando haya configurado una rama de seguimiento, puede hacer referencia a su rama ```upstream``` (ascendente) con la abreviatura ```@{upstream}``` o ```@{u}```. Entonces, si está en la branch ```master``` y está rastreando ```origin/master```, puede decir algo como en ```git merge @{u}``` lugar de ```git merge origin/master``` si lo desea.

Si desea ver qué ramas de seguimiento ha configurado, puede usar la ```-vv``` opción de ```git branch```. Esto enumerará sus branches locales con más información, incluido lo que cada branch está rastreando y si su branch local está adelante, atrás o ambos.

```
$ git branch -vv
  iss53     7e424c3 [origin/iss53: ahead 2] Add forgotten brackets
  master    1ae2a45 [origin/master] Deploy index fix
* serverfix f8674d9 [teamone/server-fix-good: ahead 3, behind 1] This should do it
  testing   5ea463a Try something new
```    
    
Entonces, aquí podemos ver que nuestra rama ```iss53``` está rastreando ```origin/iss53``` y está "adelante" en dos, lo que significa que tenemos dos confirmaciones localmente que no se envían al servidor. También podemos ver que nuestra branch ```master``` está rastreando ```origin/master``` y está actualizada. A continuación, podemos ver que nuestra rama ```serverfix``` está rastreando la rama ```server-fix-good``` en nuestro servidor ```teamone``` y está adelante en tres y detrás en uno, lo que significa que hay una confirmación en el servidor en el que no nos hemos fusionado todavía y tres confirmaciones localmente que no hemos empujado push. Finalmente, podemos ver que nuestra branch ```testing``` no está rastreando ninguna branch remota.

Es importante tener en cuenta que estos números son solo desde la última vez que buscó en cada servidor. Este comando no llega a los servidores, le informa sobre lo que ha almacenado en caché de estos servidores localmente. Si desea estar totalmente actualizado por delante y por detrás de los números, deberá buscarlos desde todos sus servidores remotos justo antes de ejecutar esto. Podrías hacer eso así:

```    
$ git fetch --all; git branch -vv
```

### Pulling (Traer)

Si bien el comando ```git fetch``` buscará todos los cambios en el servidor que aún no tiene, no modificará su directorio de trabajo en absoluto. Simplemente obtendrá los datos por usted y le permitirá fusionarlos usted mismo. Sin embargo, hay un comando llamado ```git pull``` que es esencialmente un ```git fetch``` seguido inmediatamente por un ```git merge``` en la mayoría de los casos. Si tiene una rama de seguimiento configurada como se demostró en la última sección, ya sea configurándola explícitamente o haciéndola crear para usted con los comandos ```clone``` o ```checkout```, ```git pull``` buscará qué servidor y rama está rastreando su rama actual, buscar desde ese servidor y luego intente fusionarse en esa rama remota.

Generalmente es mejor usar los comandos ```fetch``` y merge explícitamente ya que la magia de ```git pull``` a menudo puede ser confusa.

### Eliminar ramas remotas

Suponga que ha terminado con una rama remota; digamos que usted y sus colaboradores han terminado con una función y la han fusionado en la rama ```master``` de su servidor remoto (o en cualquier rama en la que se encuentre su línea de código estable). Puede eliminar una rama remota usando la opción  ```--delete``` de ```git push```. Si desea eliminar su rama ```serverfix``` del servidor, ejecute lo siguiente:

```
$ git push origin --delete serverfix
To https://github.com/schacon/simplegit
 - [deleted]         serverfix
```    
    
Básicamente, todo lo que hace es eliminar el puntero del servidor. El servidor Git generalmente mantendrá los datos allí durante un tiempo hasta que se ejecute una recolección de basura, por lo que si se eliminó accidentalmente, a menudo es fácil de recuperar.

## Git en el servidor

En este punto, debería poder realizar la mayoría de las tareas diarias para las que utilizará Git. Sin embargo, para realizar cualquier colaboración en Git, necesitará tener un repositorio de Git remoto. Aunque técnicamente puede **impulsar (push)** cambios y **extraer (pull)** cambios de los repositorios de las personas, no se recomienda hacerlo porque puede confundir con bastante facilidad en qué están trabajando si no tiene cuidado. Además, desea que sus colaboradores puedan acceder al repositorio incluso si su computadora está fuera de línea; tener un repositorio común más confiable suele ser útil. Por lo tanto, el método preferido para colaborar con alguien es configurar un repositorio intermedio al que ambos tengan acceso, y hacer **push** y **pull** de él.

Ejecutar un servidor Git es bastante sencillo. Primero, elige qué protocolos desea que admita su servidor. La primera sección de este capítulo cubrirá los protocolos disponibles y los pros y contras de cada uno. Las siguientes secciones explicarán algunas configuraciones típicas que utilizan esos protocolos y cómo hacer que su servidor funcione con ellos. Por último, repasaremos algunas opciones de alojamiento, si no le importa alojar su código en el servidor de otra persona y no quiere pasar por la molestia de configurar y mantener su propio servidor.

Si no tiene interés en ejecutar su propio servidor, puede saltar a la última sección del capítulo para ver algunas opciones para configurar una cuenta alojada y luego pasar al siguiente capítulo, donde discutimos los diversos entresijos del trabajo. en un entorno de servidor de fuente distribuido.

Un repositorio remoto es generalmente un repositorio simple: un repositorio Git que no tiene un directorio de trabajo. Debido a que el repositorio solo se usa como un punto de colaboración, no hay razón para tener una snapshot extraída en el disco; son solo los datos de Git. En los términos más simples, un repositorio simple es el contenido del directorio .git de su proyecto y nada más.

### Los protocolos

Git puede usar cuatro protocolos distintos para transferir datos: **local, HTTP, Secure Shell (SSH) y Git**. Aquí discutiremos qué son y en qué circunstancias básicas le gustaría (o no) usarlos.

### Protocolo local

El más básico es el protocolo local , en el que el repositorio remoto se encuentra en otro directorio del mismo host. Esto se usa a menudo si todos los miembros de su equipo tienen acceso a un sistema de archivos compartido, como un montaje NFS, o en el caso menos probable de que todos inicien sesión en la misma computadora. Esto último no sería ideal, porque todas sus instancias de repositorio de código residirían en la misma computadora, lo que haría mucho más probable una pérdida catastrófica.

Si tiene un sistema de archivos montado compartido, puede clonar, enviar y extraer de un repositorio local basado en archivos. Para clonar un repositorio como este, o para agregar uno como remoto a un proyecto existente, use la ruta al repositorio como URL. Por ejemplo, para clonar un repositorio local, puede ejecutar algo como esto:

```
$ git clone /srv/git/project.git
```

O puedes hacer esto:

```
$ git clone file:///srv/git/project.git
```

Git funciona de forma ligeramente diferente si especificas explícitamente ```file://``` al principio de la URL. Si solo especifica la ruta, Git intenta usar enlaces duros o copiar directamente los archivos que necesita. Si lo especifica ```file://```, Git activa los procesos que normalmente usa para transferir datos a través de una red, que generalmente es mucho menos eficiente. La razón principal para especificar el prefijo ```file://``` es si desea una copia limpia del repositorio con referencias u objetos extraños omitidos, generalmente después de una importación desde otro VCS o algo similar (consulte Git Internals para las tareas de mantenimiento). Usaremos la ruta normal aquí porque hacerlo casi siempre es más rápido.

Para agregar un repositorio local a un proyecto de Git existente, puede ejecutar algo como esto:

```
$ git remote add local_proj /srv/git/project.git
```

Luego, puede hacer **push** y **pull** de ese servidor remoto a través de su nuevo nombre de servidor remoto ```local_proj``` como si lo estuviera haciendo a través de una red.

### Los pro

Las ventajas de los repositorios basados en archivos son que son simples y utilizan los permisos de archivos existentes y el acceso a la red. Si ya tiene un sistema de archivos compartido al que todo su equipo tiene acceso, configurar un repositorio es muy fácil. Coloca la copia del repositorio simple en un lugar al que todos tienen acceso compartido y configura los permisos de lectura / escritura como lo harías con cualquier otro directorio compartido. Discutiremos cómo exportar una copia del repositorio simple para este propósito en Obtener Git en un servidor.

Esta también es una buena opción para tomar rápidamente el trabajo del repositorio de trabajo de otra persona. Si usted y un compañero de trabajo están trabajando en el mismo proyecto y quieren que compruebe algo, ejecutar un comando similar ```git pull /home/john/project``` suele ser más fácil que enviarlo a un servidor remoto y, posteriormente, recuperarlo.

### Los contras

Las desventajas de este método son que el acceso compartido es generalmente más difícil de configurar y alcanzar desde múltiples ubicaciones que el acceso básico a la red. Si desea hacer **push** desde su computadora portátil cuando está en casa, debe montar el disco remoto, que puede ser difícil y lento en comparación con el acceso basado en la red.

Es importante mencionar que esta no es necesariamente la opción más rápida si está utilizando una montura compartida de algún tipo. Un repositorio local es rápido solo si tiene acceso rápido a los datos. Un repositorio en ```NFS``` suele ser más lento que el repositorio en ```SSH``` en el mismo servidor, lo que permite que Git se ejecute desde discos locales en cada sistema.

Finalmente, este protocolo no protege el repositorio contra daños accidentales. Cada usuario tiene acceso completo de shell al directorio "remoto", y no hay nada que les impida cambiar o eliminar archivos Git internos y corromper el repositorio.

### Los protocolos HTTP

Git puede comunicarse a través de HTTP usando dos modos diferentes. Antes de Git 1.6.6, solo había una forma en que podía hacer esto, que era muy simple y generalmente de solo lectura. En la versión 1.6.6, se introdujo un nuevo protocolo más inteligente que implicaba que Git pudiera negociar inteligentemente la transferencia de datos de una manera similar a como lo hace con SSH. En los últimos años, este nuevo protocolo HTTP se ha vuelto muy popular, ya que es más sencillo para el usuario y más inteligente sobre cómo se comunica. La versión más nueva a menudo se conoce como el protocolo HTTP inteligente y la forma anterior como HTTP tonto . Primero cubriremos el protocolo HTTP inteligente más nuevo.

### HTTP inteligente

Smart HTTP funciona de manera muy similar a los protocolos SSH o Git, pero se ejecuta en puertos HTTPS estándar y puede usar varios mecanismos de autenticación HTTP, lo que significa que a menudo es más fácil para el usuario que algo como SSH, ya que puede usar cosas como autenticación de nombre de usuario/contraseña en lugar de tener para configurar claves SSH.

Probablemente se haya convertido en la forma más popular de usar Git ahora, ya que se puede configurar para que sirva de forma anónima como el protocolo ```git://```, y también se puede transferir con autenticación y cifrado como el protocolo SSH. En lugar de tener que configurar diferentes URL para estas cosas, ahora puede usar una única URL para ambas. Si intenta enviar y el repositorio requiere autenticación (que normalmente debería), el servidor puede solicitar un nombre de usuario y una contraseña. Lo mismo ocurre con el acceso de lectura.

De hecho, para servicios como GitHub, la URL que usa para ver el repositorio en línea (por ejemplo, ```https://github.com/schacon/simplegit```) es la misma URL que puede usar para clonar y, si tiene acceso, hacer **push** sobre.

### HTTP tonto

Si el servidor no responde con un servicio inteligente Git HTTP, el cliente Git intentará recurrir al protocolo HTTP Dumb más simple . El protocolo Dumb espera que el repositorio Git básico se sirva como archivos normales del servidor web. La belleza de Dumb HTTP es la simplicidad de configurarlo. Básicamente, todo lo que tienes que hacer es poner un repositorio Git bajo la raíz de tu documento HTTP y configurar un hoosk ```post-update``` específico , y listo (Ver hoosk (hosks) Git ). En ese momento, cualquier persona que pueda acceder al servidor web en el que colocas el repositorio también puede clonar tu repositorio. Para permitir el acceso de lectura a su repositorio a través de HTTP, haga algo como esto:

```
$ cd /var/www/htdocs/
$ git clone --bare /path/to/git_project gitproject.git
$ cd gitproject.git
$ mv hooks/post-update.sample hooks/post-update
$ chmod a+x hooks/post-update
```

Eso es todo. El hosk ```post-update``` que viene con Git por defecto ejecuta el comando apropiado (```git update-server-info```) para que la búsqueda y clonación HTTP funcionen correctamente. Este comando se ejecuta cuando hace **push** a este repositorio (quizás a través de SSH); luego, otras personas pueden clonar a través de algo como:

```
$ git clone https://example.com/gitproject.git
```

En este caso particular, estamos usando la ruta ```/var/www/htdocs``` que es común para las configuraciones de Apache, pero puede usar cualquier servidor web estático, simplemente coloque el repositorio en su ruta. Los datos de Git se sirven como archivos estáticos básicos (consulte el capítulo Internals de Git para obtener detalles sobre cómo se sirven exactamente).

En general, elegiría ejecutar un servidor HTTP inteligente de lectura/escritura o simplemente tener los archivos accesibles como de solo lectura de la manera tonta. Es raro ejecutar una combinación de los dos servicios.

### Los pro

Nos concentraremos en las ventajas de la versión inteligente del protocolo HTTP.

La simplicidad de tener una única URL para todos los tipos de acceso y que el servidor solicite solo cuando se necesita autenticación hace que las cosas sean muy fáciles para el usuario final. Poder autenticarse con un nombre de usuario y contraseña también es una gran ventaja sobre SSH, ya que los usuarios no tienen que generar claves SSH localmente y cargar su clave pública en el servidor antes de poder interactuar con él. Para usuarios menos sofisticados, o usuarios en sistemas donde SSH es menos común, esta es una gran ventaja en usabilidad. También es un protocolo muy rápido y eficiente, similar al SSH.

También puede servir sus repositorios de solo lectura a través de HTTPS, lo que significa que puede cifrar la transferencia de contenido; o puede ir tan lejos como para hacer que los clientes usen certificados SSL firmados específicos.

Otra cosa interesante es que HTTP y HTTPS son protocolos de uso tan común que los cortafuegos corporativos a menudo se configuran para permitir el tráfico a través de sus puertos.

### Los contras

Git sobre HTTPS puede ser un poco más complicado de configurar en comparación con SSH en algunos servidores. Aparte de eso, hay muy pocas ventajas que otros protocolos tienen sobre Smart HTTP para servir contenido de Git.

Si está utilizando HTTP para la inserción autenticada, proporcionar sus credenciales a veces es más complicado que usar claves a través de SSH. Sin embargo, existen varias herramientas de almacenamiento en caché de credenciales que puede usar, incluido el acceso a Llaveros en macOS y Credential Manager en Windows, para que esto sea bastante sencillo. Lea Almacenamiento de credenciales para ver cómo configurar el almacenamiento en caché de contraseñas HTTP seguras en su sistema.

### El protocolo SSH

Un protocolo de transporte común para Git cuando el autohosting se realiza a través de SSH. Esto se debe a que el acceso SSH a los servidores ya está configurado en la mayoría de los lugares, y si no lo está, es fácil de hacer. SSH también es un protocolo de red autenticado y, debido a que es ubicuo, generalmente es fácil de configurar y usar.

Para clonar un repositorio de Git a través de SSH, puede especificar una URL ```ssh://``` como esta:

```
$ git clone ssh://[user@]server/project.git
```

O puede usar la sintaxis similar a scp más corta para el protocolo SSH:

```
$ git clone [user@]server:project.git
```

En ambos casos anteriores, si no especifica el nombre de usuario opcional, Git asume el usuario con el que está conectado actualmente.

### Los pro

Las ventajas de usar SSH son muchas. Primero, SSH es relativamente fácil de configurar: los demonios SSH son comunes, muchos administradores de red tienen experiencia con ellos y muchas distribuciones de SO están configuradas con ellos o tienen herramientas para administrarlos. A continuación, el acceso a través de SSH es seguro: toda la transferencia de datos está cifrada y autenticada. Por último, al igual que los protocolos HTTPS, Git y Local, SSH es eficiente, lo que hace que los datos sean lo más compactos posible antes de transferirlos.

### Los contras

El aspecto negativo de SSH es que no admite el acceso anónimo a su repositorio de Git. Si está utilizando SSH, las personas deben tener acceso SSH a su máquina, incluso en una capacidad de solo lectura, lo que no hace que SSH sea propicio para proyectos de código abierto para los que las personas simplemente deseen clonar su repositorio para examinarlo. Si lo está utilizando solo dentro de su red corporativa, SSH puede ser el único protocolo con el que debe lidiar. Si desea permitir el acceso anónimo de solo lectura a sus proyectos y también desea usar SSH, tendrá que configurar SSH para que lo transfiera, pero algo más para que otros lo obtengan.

### El protocolo Git

Finalmente, tenemos el protocolo Git. Este es un demonio especial que viene empaquetado con Git; escucha en un puerto dedicado (9418) que proporciona un servicio similar al protocolo SSH, pero sin absolutamente ninguna autenticación. Para que un repositorio se sirva a través del protocolo Git, debe crear un archivo ```git-daemon-export-ok``` (el daemon no servirá a un repositorio sin ese archivo en él) pero, aparte de eso, no hay seguridad. O el repositorio de Git está disponible para que todos puedan clonarlo, o no lo está. Esto significa que, por lo general, no hay necesidad de hacer **push** sobre este protocolo. Puede habilitar el acceso de inserción pero, dada la falta de autenticación, cualquier persona en Internet que encuentre la URL de su proyecto podría ingresar a ese proyecto. Baste decir que esto es raro.

### Los pro

El protocolo Git es a menudo el protocolo de transferencia de red más rápido disponible. Si está sirviendo mucho tráfico para un proyecto público o sirviendo a un proyecto muy grande que no requiere autenticación de usuario para el acceso de lectura, es probable que desee configurar un daemon Git para servir su proyecto. Utiliza el mismo mecanismo de transferencia de datos que el protocolo SSH pero sin la sobrecarga de cifrado y autenticación.

### Los contras

La desventaja del protocolo Git es la falta de autenticación. Por lo general, no es deseable que el protocolo Git sea el único acceso a su proyecto. Por lo general, lo emparejará con acceso SSH o HTTPS para los pocos desarrolladores que tienen acceso **push** (escritura) y que todos los demás lo usan ```git://``` para acceso de solo lectura. También es probablemente el protocolo más difícil de configurar. Se debe seguir su propio deamon, lo que requiere configuración ```xinetdo``` o ```systemd``` similares, que no siempre es un paseo por el parque. También requiere acceso de firewall al puerto 9418, que no es un puerto estándar que los firewalls corporativos siempre permiten. Detrás de los grandes cortafuegos corporativos, este puerto oscuro suele estar bloqueado.

### Obtener Git en un servidor
Ahora cubriremos la configuración de un servicio Git que ejecute estos protocolos en su propio servidor.

**Nota**

Aquí demostraremos los comandos y pasos necesarios para realizar instalaciones básicas y simplificadas en un servidor basado en Linux, aunque también es posible ejecutar estos servicios en servidores macOS o Windows. La configuración real de un servidor de producción dentro de su infraestructura ciertamente implicará diferencias en las medidas de seguridad o las herramientas del sistema operativo, pero es de esperar que esto le dé una idea general de lo que está involucrado.

Para configurar inicialmente cualquier servidor Git, debe exportar un repositorio existente a un nuevo repositorio desnudo, un repositorio que no contiene un directorio de trabajo. Por lo general, esto es sencillo de hacer. Para clonar su repositorio para crear un nuevo repositorio desnudo, ejecute el comando clonar con la opción ```--bare```. Por convención, los nombres de directorio de repositorios simples terminan con el sufijo ```.git```, así:

```
$ git clone --bare my_project my_project.git
Cloning into bare repository 'my_project.git'...
done.
```

Ahora debería tener una copia de los datos del directorio Git en su directorio ```my_project.git```.

Esto es aproximadamente equivalente a algo como:

```
$ cp -Rf my_project/.git my_project.git
```

Hay un par de diferencias menores en el archivo de configuración pero, para su propósito, esto es casi lo mismo. Toma el repositorio de Git por sí solo, sin un directorio de trabajo, y crea un directorio específicamente solo para él.

### Colocación del repositorio desnudo en un servidor

Ahora que tiene una copia simple de su repositorio, todo lo que necesita hacer es colocarlo en un servidor y configurar sus protocolos. Supongamos que ha configurado un servidor llamado ```git.example.com``` al que tiene acceso SSH y desea almacenar todos sus repositorios de Git en el directorio ```/srv/git```. Suponiendo que exista ```/srv/git``` en ese servidor, puede configurar su nuevo repositorio copiando su repositorio simple sobre:

```
$ scp -r my_project.git user@git.example.com:/srv/git
```

En este punto, otros usuarios que tienen acceso de lectura basado en SSH al directorio ```/srv/git``` en ese servidor pueden clonar su repositorio ejecutando:

```
$ git clone user@git.example.com:/srv/git/my_project.git
```

Si un usuario ingresa por SSH en un servidor y tiene acceso de escritura al directorio ```/srv/git/my_project.git```, también tendrá acceso de inserción automáticamente.

Git agregará automáticamente permisos de escritura de grupo a un repositorio correctamente si ejecuta el comando ```git init``` con la opción ```--shared```. Tenga en cuenta que al ejecutar este comando, no destruirá ninguna confirmación, referencia, etc. en el proceso.

```
$ ssh user@git.example.com
$ cd /srv/git/my_project.git
$ git init --bare --shared
```

Verá lo fácil que es tomar un repositorio de Git, crear una versión básica y colocarlo en un servidor al que usted y sus colaboradores tienen acceso SSH. Ahora está listo para colaborar en el mismo proyecto.

Es importante tener en cuenta que esto es literalmente todo lo que necesita hacer para ejecutar un servidor Git útil al que varias personas tienen acceso: simplemente agregue cuentas SSH en un servidor y coloque un repositorio simple en algún lugar donde todos esos usuarios hayan leído y escrito el acceso Está listo para comenzar, no se necesita nada más.

En las siguientes secciones, verá cómo expandirse a configuraciones más sofisticadas. Esta discusión incluirá no tener que crear cuentas de usuario para cada usuario, agregar acceso de lectura público a los repositorios, configurar interfaces de usuario web y más. Sin embargo, tenga en cuenta que para colaborar con un par de personas en un proyecto privado, todo lo que necesita es un servidor SSH y un repositorio simple.

### Pequeñas configuraciones

Si tiene un equipo pequeño o simplemente está probando Git en su organización y solo tiene unos pocos desarrolladores, las cosas pueden ser simples para usted. Uno de los aspectos más complicados de configurar un servidor Git es la administración de usuarios. Si desea que algunos repositorios sean de solo lectura para ciertos usuarios y de lectura/escritura para otros, el acceso y los permisos pueden ser un poco más difíciles de organizar.

### Acceso SSH

Si tiene un servidor al que todos sus desarrolladores ya tienen acceso SSH, generalmente es más fácil configurar su primer repositorio allí, porque casi no tiene que hacer ningún trabajo (como cubrimos en la última sección). Si desea permisos de tipo de control de acceso más complejos en sus repositorios, puede manejarlos con los permisos normales del sistema de archivos del sistema operativo de su servidor.

Si desea colocar sus repositorios en un servidor que no tiene cuentas para todos los miembros de su equipo a quienes desea otorgar acceso de escritura, debe configurar el acceso SSH para ellos. Suponemos que si tiene un servidor con el que hacer esto, ya tiene un servidor SSH instalado, y así es como está accediendo al servidor.

Hay algunas formas en las que puede dar acceso a todos los miembros de su equipo. La primera es configurar cuentas para todos, lo cual es sencillo pero puede resultar engorroso. Es posible que no desee ejecutar ```adduser``` (o la posible alternativa ```useradd```) y tenga que establecer contraseñas temporales para cada nuevo usuario.

Un segundo método es crear una sola cuenta de usuario 'git' en la máquina, pedirle a cada usuario que tenga acceso de escritura que le envíe una clave pública SSH y agregar esa clave al archivo ```~/.ssh/authorized_keys```  de esa nueva cuenta 'git'. En ese momento, todos podrán acceder a esa máquina a través de la cuenta 'git'. Esto no afecta los datos de confirmación de ninguna manera: el usuario SSH con el que se conecta no afecta las confirmaciones que ha registrado.

Otra forma de hacerlo es hacer que su servidor SSH se autentique desde un servidor LDAP o alguna otra fuente de autenticación centralizada que ya haya configurado. Siempre que cada usuario pueda obtener acceso de shell en la máquina, cualquier mecanismo de autenticación SSH que se le ocurra debería funcionar.

### Generación de su clave pública SSH

Muchos servidores Git se autentican mediante claves públicas SSH. Para proporcionar una clave pública, cada usuario de su sistema debe generar una si aún no la tiene. Este proceso es similar en todos los sistemas operativos. Primero, debe verificar para asegurarse de que aún no tiene una clave. De forma predeterminada, las claves SSH de un usuario se almacenan en el directorio ```~/.ssh``` de ese usuario. Puede verificar fácilmente si ya tiene una clave yendo a ese directorio y enumerando el contenido:

```
$ cd ~/.ssh
$ ls
authorized_keys2  id_dsa       known_hosts
config            id_dsa.pub
```

Está buscando un par de archivos con un nombre similar ```id_dsa``` o ```id_rsa``` y un archivo coincidente con una extensión ```.pub```. El archivo ```.pub``` es su clave pública y el otro archivo es la clave privada correspondiente. Si no tiene estos archivos (o ni siquiera tiene un directorio ```.ssh```), puede crearlos ejecutando un programa llamado ```ssh-keygen```, que se proporciona con el paquete SSH en los sistemas Linux/macOS y viene con Git para Windows:

```
$ ssh-keygen -o
Generating public/private rsa key pair.
Enter file in which to save the key (/home/schacon/.ssh/id_rsa):
Created directory '/home/schacon/.ssh'.
Enter passphrase (empty for no passphrase):
Enter same passphrase again:
Your identification has been saved in /home/schacon/.ssh/id_rsa.
Your public key has been saved in /home/schacon/.ssh/id_rsa.pub.
The key fingerprint is:
d0:82:24:8e:d7:f1:bb:9b:33:53:96:93:49:da:9b:e3 schacon@mylaptop.local
```

Primero confirma dónde desea guardar la clave (```.ssh/id_rsa```), y luego le pide dos veces una frase de contraseña, que puede dejar en blanco si no desea escribir una contraseña cuando usa la clave. Sin embargo, si usa una contraseña, asegúrese de agregar la opción ```-o```; guarda la clave privada en un formato que es más resistente al descifrado de contraseñas por fuerza bruta que el formato predeterminado. También puede usar la herramienta ```ssh-agent``` para evitar tener que ingresar la contraseña cada vez.

Ahora, cada usuario que haga esto tiene que enviarle su clave pública a usted o a quien esté administrando el servidor Git (asumiendo que está usando una configuración de servidor SSH que requiere claves públicas). Todo lo que tienen que hacer es copiar el contenido del archivo ```.pub``` y enviarlo por correo electrónico. Las claves públicas se parecen a esto:

```
$ cat ~/.ssh/id_rsa.pub
ssh-rsa AAAAB3NzaC1yc2EAAAABIwAAAQEAklOUpkDHrfHY17SbrmTIpNLTGK9Tjom/BWDSU
GPl+nafzlHDTYW7hdI4yZ5ew18JH4JW9jbhUFrviQzM7xlELEVf4h9lFX5QVkbPppSwg0cda3
Pbv7kOdJ/MTyBlWXFCR+HAo3FXRitBqxiX1nKhXpHAZsMciLq8V6RjsNAQwdsdMFvSlVK/7XA
t3FaoJoAsncM1Q9x5+3V0Ww68/eIFmb1zuUFljQJKprrX88XypNDvjYNby6vw/Pb0rwert/En
mZ+AW4OZPnTPI89ZPmVMLuayrD2cE86Z/il8b+gw3r3+1nKatmIkjn2so1d01QraTlMqVSsbx
NrRFi9wrf+M7Q== schacon@mylaptop.local
```

Para obtener un tutorial más detallado sobre cómo crear una clave SSH en varios sistemas operativos, consulte la guía de GitHub sobre claves SSH en https://docs.github.com/en/github/authenticating-to-github/generating-a-new-ssh-key-and-add-it-to-the-ssh-agent .



### Configurar el servidor

Veamos cómo configurar el acceso SSH en el lado del servidor. En este ejemplo, utilizará el método ```authorized_keys```  para autenticar a sus usuarios. También asumimos que está ejecutando una distribución estándar de Linux como Ubuntu.

**Nota**

Gran parte de lo que se describe aquí se puede automatizar utilizando el comando ```ssh-copy-id```, en lugar de copiar e instalar manualmente las claves públicas.

Primero, crea una cuenta git de usuario y un directorio ```.ssh ```para ese usuario.

```
$ sudo adduser git
$ su git
$ cd
$ mkdir .ssh && chmod 700 .ssh
$ touch .ssh/authorized_keys && chmod 600 .ssh/authorized_keys
```

A continuación, debe agregar algunas claves públicas SSH de desarrollador al archivo ```authorized_keys``` para el usuario ```git```. Supongamos que tiene algunas claves públicas confiables y las ha guardado en archivos temporales. Nuevamente, las claves públicas se ven así:

```
$ cat /tmp/id_rsa.john.pub
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCB007n/ww+ouN4gSLKssMxXnBOvf9LGt4L
ojG6rs6hPB09j9R/T17/x4lhJA0F3FR1rP6kYBRsWj2aThGw6HXLm9/5zytK6Ztg3RPKK+4k
Yjh6541NYsnEAZuXz0jTTyAUfrtU3Z5E003C4oxOj6H0rfIF1kKI9MAQLMdpGW1GYEIgS9Ez
Sdfd8AcCIicTDWbqLAcU4UpkaX8KyGlLwsNuuGztobF8m72ALC/nLF6JLtPofwFBlgc+myiv
O7TCUSBdLQlgMVOFq1I2uPWQOkOWQAHukEOmfjy2jctxSDBQ220ymjaNsHT4kgtZg2AYYgPq
dAv8JggJICUvax2T9va5 gsg-keypair
```

Simplemente añádalos al archivo git del usuario ```authorized_keys``` en su directorio ```.ssh```:

```
$ cat /tmp/id_rsa.john.pub >> ~/.ssh/authorized_keys
$ cat /tmp/id_rsa.josie.pub >> ~/.ssh/authorized_keys
$ cat /tmp/id_rsa.jessica.pub >> ~/.ssh/authorized_keys
```

Ahora, puede configurar un repositorio vacío para ellos ejecutando ```git init``` la opción ```--bare```, que inicializa el repositorio sin un directorio de trabajo:

```
$ cd /srv/git
$ mkdir project.git
$ cd project.git
$ git init --bare
Initialized empty Git repository in /srv/git/project.git/
```

Luego, John, Josie o Jessica pueden enviar la primera versión de su proyecto a ese repositorio agregándola como servidor remoto y subiendo una rama. Tenga en cuenta que alguien debe ingresar a la máquina y crear un repositorio simple cada vez que desee agregar un proyecto. Usemos ```gitserver``` como nombre de host del servidor en el que ha configurado su usuario ```git``` y repositorio. Si lo está ejecutando internamente y configura DNS para que ```gitserver```  apunte a ese servidor, entonces puede usar los comandos tal como están (asumiendo que ```myproject``` es un proyecto existente con archivos):

```
# on John's computer
$ cd myproject
$ git init
$ git add .
$ git commit -m 'Initial commit'
$ git remote add origin git@gitserver:/srv/git/project.git
$ git push origin master
```

En este punto, los demás pueden clonarlo y volver a subir los cambios con la misma facilidad:

```
$ git clone git@gitserver:/srv/git/project.git
$ cd project
$ vim README
$ git commit -am 'Fix for README file'
$ git push origin master
```

Con este método, puede poner rápidamente en funcionamiento un servidor Git de lectura/escritura para un puñado de desarrolladores.

Debe tener en cuenta que actualmente todos estos usuarios también pueden iniciar sesión en el servidor y obtener un ```shell``` como usuario ```git```. Si desea restringir eso, tendrá que cambiar el ```shell``` a otra cosa en el archivo ```/etc/passwd```.

Puede restringir fácilmente la cuenta ```git``` de usuario a solo actividades relacionadas con ```Git``` con una herramienta de ```shell``` limitada llamada ```git-shell``` que viene con ```Git```. Si configura esto como el ```gitshell``` de inicio de sesión de la cuenta de usuario, entonces esa cuenta no puede tener acceso de ```shell``` normal a su servidor. Para usar esto, especifique ```git-shell``` en lugar de ```bash``` o ```csh``` para el ```shell``` de inicio de sesión de esa cuenta. Para hacerlo, primero debe agregar la ruta completa del comando ```git-shell``` a ```/etc/shells``` si aún no está allí:

```
$ cat /etc/shells   # see if git-shell is already in there. If not...
$ which git-shell   # make sure git-shell is installed on your system.
$ sudo -e /etc/shells  # and add the path to git-shell from last command
```

Ahora puede editar el shell para un usuario usando ```chsh <username> -s <shell>```:

```
$ sudo chsh git -s $(which git-shell)
```

Ahora, el usuario git aún puede usar la conexión SSH para empujar **(push)** y extraer **(pull)** repositorios de Git, pero no puede acceder a la máquina. Si lo intenta, verá un rechazo de inicio de sesión como este:

```
$ ssh git@gitserver
fatal: Interactive git shell is not enabled.
hint: ~/git-shell-commands should exist and have read and execute access.
Connection to gitserver closed.
```

En este punto, los usuarios aún pueden usar el reenvío de puertos ```SSH``` para acceder a cualquier host al que pueda acceder el servidor ```git```. Si desea evitar eso, puede editar el archivo ```authorized_keys``` y anteponer las siguientes opciones a cada tecla que desea restringir:

```
no-port-forwarding,no-X11-forwarding,no-agent-forwarding,no-pty
```

El resultado debería verse así:

```
$ cat ~/.ssh/authorized_keys
no-port-forwarding,no-X11-forwarding,no-agent-forwarding,no-pty ssh-rsa
AAAAB3NzaC1yc2EAAAADAQABAAABAQCB007n/ww+ouN4gSLKssMxXnBOvf9LGt4LojG6rs6h
PB09j9R/T17/x4lhJA0F3FR1rP6kYBRsWj2aThGw6HXLm9/5zytK6Ztg3RPKK+4kYjh6541N
YsnEAZuXz0jTTyAUfrtU3Z5E003C4oxOj6H0rfIF1kKI9MAQLMdpGW1GYEIgS9EzSdfd8AcC
IicTDWbqLAcU4UpkaX8KyGlLwsNuuGztobF8m72ALC/nLF6JLtPofwFBlgc+myivO7TCUSBd
LQlgMVOFq1I2uPWQOkOWQAHukEOmfjy2jctxSDBQ220ymjaNsHT4kgtZg2AYYgPqdAv8JggJ
ICUvax2T9va5 gsg-keypair

no-port-forwarding,no-X11-forwarding,no-agent-forwarding,no-pty ssh-rsa
AAAAB3NzaC1yc2EAAAADAQABAAABAQDEwENNMomTboYI+LJieaAY16qiXiH3wuvENhBG...
```

Ahora los comandos de red de Git seguirán funcionando bien, pero los usuarios no podrán obtener un ```shell```. Como indica el resultado, también puede configurar un directorio en el directorio ```git``` de inicio del usuario que personalice un poco el comando  ```git-shell```. Por ejemplo, puede restringir los comandos de Git que aceptará el servidor o puede personalizar el mensaje que ven los usuarios si intentan ```SSH``` de esa manera. Ejecute ```git help shell``` para obtener más información sobre cómo personalizar el ```shell```.

### Git Daemon

A continuación, configuraremos un demonio (daemon) que sirva repositorios utilizando el protocolo "Git". Esta es una opción común para un acceso rápido y no autenticado a sus datos de Git. Recuerde que, dado que este no es un servicio autenticado, todo lo que sirva a través de este protocolo es público dentro de su red.

Si está ejecutando esto en un servidor fuera de su firewall, debe usarse solo para proyectos que son públicamente visibles para el mundo. Si el servidor en el que lo está ejecutando está dentro de su firewall, puede usarlo para proyectos a los que una gran cantidad de personas o computadoras (integración continua o servidores de compilación) tienen acceso de solo lectura, cuando usted no desea tener para agregar una clave SSH para cada uno.

En cualquier caso, el protocolo Git es relativamente fácil de configurar. Básicamente, debe ejecutar este comando de forma demonizada (daemonized):

```
$ git daemon --reuseaddr --base-path=/srv/git/ /srv/git/
```

La opción ```--reuseadd``` rermite que el servidor se reinicie sin esperar a que se agoten las conexiones antiguas, mientras que la opción ```--base-path``` permite a las personas clonar proyectos sin especificar la ruta completa, y la ruta al final le dice al daemon Git dónde buscar repositorios para exportar. Si está ejecutando un firewall, también deberá perforar un agujero en el puerto 9418 en la caja en la que está configurando esto.

Puede demonizar este proceso de varias formas, dependiendo del sistema operativo que esté ejecutando.

Dado que ```systemd``` es el sistema de inicio más común entre las distribuciones modernas de Linux, puede usarlo para ese propósito. Simplemente coloque un archivo ```/etc/systemd/system/git-daemon.service``` con estos contenidos:

```
[Unit]
Description=Start Git Daemon

[Service]
ExecStart=/usr/bin/git daemon --reuseaddr --base-path=/srv/git/ /srv/git/

Restart=always
RestartSec=500ms

StandardOutput=syslog
StandardError=syslog
SyslogIdentifier=git-daemon

User=git
Group=git

[Install]
WantedBy=multi-user.target
```

Es posible que haya notado que el daemon de Git se inicia aquí como git grupo y como usuario. Modifíquelo para que se ajuste a sus necesidades y asegúrese de que el usuario proporcionado exista en el sistema. Además, verifique que el binario de Git esté ubicado en ```/usr/bin/git``` y cambie la ruta si es necesario.

Finalmente, ejecutará ```systemctl enable git-daemon``` para iniciar automáticamente el servicio al arrancar y podrá iniciar y detener el servicio con, respectivamente, ```systemctl start git-daemon``` y ```systemctl stop git-daemon```.

En otros sistemas, es posible que desee utilizar ```xinetdun script``` en su sistema ```sysvinit``` o algo más, siempre que obtenga ese comando demonizado y observado de alguna manera.

A continuación, debe indicarle a Git a qué repositorios permitir el acceso basado en el servidor de Git no autenticado. Puede hacer esto en cada repositorio creando un archivo llamado ```git-daemon-export-ok```.

```
$ cd /path/to/project.git
$ touch git-daemon-export-ok
```

La presencia de ese archivo le dice a Git que está bien entregar este proyecto sin autenticación.

### HTTP inteligente

Ahora tenemos acceso autenticado a través de SSH y acceso no autenticado a través ```git://```, pero también hay un protocolo que puede hacer ambas cosas al mismo tiempo. La configuración de Smart HTTP consiste básicamente en habilitar un script CGI que se proporciona con Git llamado ```git-http-backend``` en el servidor. Este CGI leerá la ruta y los encabezados enviados por un ```git fetch``` o ```git push``` hacia una URL HTTP y determinará si el cliente puede comunicarse a través de HTTP (lo cual es cierto para cualquier cliente desde la versión 1.6.6). Si el CGI ve que el cliente es inteligente, se comunicará inteligentemente con él; de lo contrario, volverá al comportamiento tonto (por lo que es compatible con versiones anteriores para lecturas con clientes más antiguos).

Repasemos una configuración muy básica. Lo configuraremos con Apache como servidor CGI. Si no tiene la configuración de Apache, puede hacerlo en una caja de Linux con algo como esto:

```
$ sudo apt-get install apache2 apache2-utils
$ a2enmod cgi alias env
```

Esto también permite a los  módulos ```mod_cgi, mod_alias y mod_env```, que son todos necesarios para que esto funcione correctamente.

También deberá configurar el grupo de usuarios de Unix de los directorios ```/srv/git``` para que su servidor web  ```www-data``` pueda acceder a los repositorios de lectura y escritura, porque la instancia de Apache que ejecuta el script CGI se ejecutará (de forma predeterminada) como ese usuario:

```
$ chgrp -R www-data /srv/git
```

A continuación, debemos agregar algunas cosas a la configuración de Apache para ejecutar ```git-http-backen``` del controlador para cualquier cosa que se encuentre en la ruta ```/git``` de su servidor web.

```
SetEnv GIT_PROJECT_ROOT /srv/git
SetEnv GIT_HTTP_EXPORT_ALL
ScriptAlias /git/ /usr/lib/git-core/git-http-backend/
```

Si omite la variable de entorno ```GIT_HTTP_EXPORT_ALL``` , Git solo servirá a los clientes no autenticados los repositorios con el archivo ```git-daemon-export-ok``` en ellos, tal como lo hizo el demonio de Git.

Finalmente, querrá decirle a Apache que permita las solicitudes ```git-http-backend``` y haga que las escrituras se autentiquen de alguna manera, posiblemente con un bloque de autenticación como este:

```
<Files "git-http-backend">
    AuthType Basic
    AuthName "Git Access"
    AuthUserFile /srv/git/.htpasswd
    Require expr !(%{QUERY_STRING} -strmatch '*service=git-receive-pack*' || %{REQUEST_URI} =~ m#/git-receive-pack$#)
    Require valid-user
</Files>
```

Eso requerirá que cree un archivo ```.htpasswd``` que contenga las contraseñas de todos los usuarios válidos. A continuación, se muestra un ejemplo de cómo agregar un usuario ```"schacon"``` al archivo:

```
$ htpasswd -c /srv/git/.htpasswd schacon
```

Hay muchas formas de hacer que Apache autentique a los usuarios, tendrá que elegir e implementar una de ellas. Este es solo el ejemplo más simple que se nos ocurrió. Es casi seguro que también querrá configurar esto a través de SSL para que todos estos datos estén encriptados.

No queremos ir demasiado lejos en la trampa del conejo de las especificaciones de configuración de Apache, ya que bien podría estar usando un servidor diferente o tener diferentes necesidades de autenticación. La idea es que Git venga con un CGI llamado ```git-http-backend``` que cuando se invoca hará toda la negociación para enviar y recibir datos a través de HTTP. No implementa ninguna autenticación en sí, pero eso se puede controlar fácilmente en la capa del servidor web que lo invoca. Puede hacer esto con casi cualquier servidor web compatible con CGI, así que elija el que mejor conozca.

**Nota**

Para obtener más información sobre cómo configurar la autenticación en Apache, consulte los documentos de Apache aquí: https://httpd.apache.org/docs/current/howto/auth.html

### GitWeb

Ahora que tiene acceso básico de lectura/escritura y solo lectura a su proyecto, es posible que desee configurar un visualizador simple basado en la web. Git viene con un script CGI llamado GitWeb que a veces se usa para esto.

![](Images/img28.png)
Figura 49. La interfaz de usuario basada en web de GitWeb

Si desea ver cómo se vería GitWeb para su proyecto, Git viene con un comando para iniciar una instancia temporal si tiene un servidor web liviano en su sistema como ```lighttpd``` o ```webrick```. En máquinas Linux, a menudo ```lighttpd``` se instala, por lo que es posible que pueda ejecutarlo escribiendo ```git instaweb``` en el directorio de su proyecto. Si está ejecutando una Mac, ```Leopard``` viene preinstalado con Ruby, por lo que ```webrick``` puede ser su mejor opción. Para comenzar con ```instaweb``` un controlador que no sea ```lighttpd```, puede ejecutarlo con la opción ```--httpd```.

```
$ git instaweb --httpd=webrick
[2009-02-21 10:02:21] INFO  WEBrick 1.3.1
[2009-02-21 10:02:21] INFO  ruby 1.8.6 (2008-03-03) [universal-darwin9.0]
```

Eso inicia un servidor HTTPD en el puerto 1234 y luego inicia automáticamente un navegador web que se abre en esa página. Es bastante fácil de tu parte. Cuando haya terminado y desee apagar el servidor, puede ejecutar el mismo comando con la opción ```--stop```:

```
$ git instaweb --httpd=webrick --stop
```

Si desea ejecutar la interfaz web en un servidor todo el tiempo para su equipo o para un proyecto de código abierto que está alojando, deberá configurar el ```script CGI``` para que lo sirva su servidor web normal. Algunas distribuciones de Linux tienen un paquete ```gitweb``` que puede instalar a través de ```apt``` o ```dnf```, por lo que es posible que desee probarlo primero. Pasaremos por la instalación de ```GitWeb``` manualmente muy rápidamente. Primero, debe obtener el código fuente de Git, que viene con ```GitWeb```, y generar el ```script CGI``` personalizado:

```
$ git clone git://git.kernel.org/pub/scm/git/git.git
$ cd git/
$ make GITWEB_PROJECTROOT="/srv/git" prefix=/usr gitweb
    SUBDIR gitweb
    SUBDIR ../
make[2]: `GIT-VERSION-FILE' is up to date.
    GEN gitweb.cgi
    GEN static/gitweb.js
$ sudo cp -Rf gitweb /var/www/
```

Tenga en cuenta que tiene que decirle al comando dónde encontrar sus repositorios Git con la variable ```GITWEB_PROJECTROOT```. Ahora, debe hacer que Apache use ```CGI``` para ese ```script```, para lo cual puede agregar un ```VirtualHost```:

```
<VirtualHost *:80>
    ServerName gitserver
    DocumentRoot /var/www/gitweb
    <Directory /var/www/gitweb>
        Options +ExecCGI +FollowSymLinks +SymLinksIfOwnerMatch
        AllowOverride All
        order allow,deny
        Allow from all
        AddHandler cgi-script cgi
        DirectoryIndex gitweb.cgi
    </Directory>
</VirtualHost>
```

Una vez más, GitWeb se puede servir con cualquier servidor web compatible con ```CGI``` o ```Perl```; si prefiere usar otra cosa, no debería ser difícil de configurar. En este punto, debería poder visitar http://gitserver/ para ver sus repositorios en línea.

### GitLab

Sin embargo, GitWeb es bastante simplista. Si está buscando un servidor Git moderno y con todas las funciones, existen varias soluciones de código abierto que puede instalar en su lugar. Como GitLab es uno de los más populares, cubriremos su instalación y uso como ejemplo. Esto es más difícil que la opción GitWeb y requerirá más mantenimiento, pero es una opción con todas las funciones.

### Instalación

GitLab es una aplicación web respaldada por una base de datos, por lo que su instalación es más complicada que la de otros servidores Git. Afortunadamente, este proceso está bien documentado y respaldado. GitLab recomienda encarecidamente instalar GitLab en su servidor a través del paquete oficial ```Omnibus GitLab```.

Las otras opciones de instalación son:

* Gráfico de GitLab Helm, para usar con Kubernetes.

* Paquetes de GitLab Dockerized para usar con Docker.

* De los archivos fuente.

* Proveedor de nube como AWS, Google Cloud Platform, Azure, OpenShift y Digital Ocean.

Para obtener más información, lea el archivo Léame de GitLab Community Edition (CE).

### Administración

Se accede a la interfaz de administración de GitLab a través de la web. Simplemente apunte su navegador al nombre de host o dirección IP donde está instalado GitLab e inicie sesión como usuario administrador. El nombre de usuario predeterminado es ```admin@local.host```, y la contraseña predeterminada es ```5iveL!fe``` (que debe cambiar de inmediato). Una vez que haya iniciado sesión, haga clic en el icono "Área de administración" en el menú en la parte superior derecha.

![](Images/img29.png)
Figura 50. El elemento "Área de administración" en el menú de GitLab

### Usuarios

Todo el que use su servidor GitLab debe tener una cuenta de usuario. Las cuentas de usuario son bastante simples, contienen principalmente información personal adjunta a los datos de inicio de sesión. Cada cuenta de usuario tiene un espacio de nombres, que es una agrupación lógica de proyectos que pertenecen a ese usuario. Si el usuario jane tuviera un proyecto llamado proyecto, la URL de ese proyecto sería http://server/jane/project.

![](Images/img30.png)
Figura 51. La pantalla de administración de usuarios de GitLab

Puede eliminar una cuenta de usuario de dos formas: "Bloquear" a un usuario evita que inicie sesión en la instancia de GitLab, pero se conservarán todos los datos del espacio de nombres de ese usuario y las confirmaciones firmadas con la dirección de correo electrónico de ese usuario seguirán vinculando a su perfil.

“Destruir” a un usuario, por otro lado, lo elimina por completo de la base de datos y del sistema de archivos. Se eliminan todos los proyectos y datos de su espacio de nombres, y también se eliminarán los grupos que posean. Obviamente, esta es una acción mucho más permanente y destructiva, y rara vez la necesitará.

### Grupos

Un grupo de GitLab es una colección de proyectos, junto con datos sobre cómo los usuarios pueden acceder a esos proyectos. Cada grupo tiene un espacio de nombres de proyecto (de la misma manera que lo hacen los usuarios), por lo que si la capacitación grupal tiene materiales del proyecto, su URL sería http://server/training/materials.

![](Images/img31.png)
Figura 52. La pantalla de administración del grupo de GitLab

Cada grupo está asociado con una cantidad de usuarios, cada uno de los cuales tiene un nivel de permisos para los proyectos del grupo y el grupo en sí. Estos van desde "Invitado" (solo problemas y chat) hasta "Propietario" (control total del grupo, sus miembros y sus proyectos). Los tipos de permisos son demasiado numerosos para enumerarlos aquí, pero GitLab tiene un enlace útil en la pantalla de administración.

### Proyectos

Un proyecto de GitLab corresponde aproximadamente a un único repositorio de Git. Cada proyecto pertenece a un solo espacio de nombres, ya sea un usuario o un grupo. Si el proyecto pertenece a un usuario, el propietario del proyecto tiene control directo sobre quién tiene acceso al proyecto; si el proyecto pertenece a un grupo, se aplicarán los permisos de nivel de usuario del grupo.

Cada proyecto tiene un nivel de visibilidad, que controla quién tiene acceso de lectura a las páginas y al repositorio de ese proyecto. Si un proyecto es privado , el propietario del proyecto debe otorgar acceso explícitamente a usuarios específicos. Un proyecto interno es visible para cualquier usuario que haya iniciado sesión, y un proyecto público es visible para todos. Tenga en cuenta que esto controla tanto el acceso git fetch como el acceso a la interfaz de usuario web para ese proyecto.

### Hooks

GitLab incluye soporte para Hooks, tanto a nivel de proyecto como de sistema. Para cualquiera de estos, el servidor de GitLab realizará un HTTP POST con un JSON descriptivo siempre que ocurran eventos relevantes. Esta es una excelente manera de conectar sus repositorios de Git y la instancia de GitLab al resto de su automatización de desarrollo, como servidores de CI, salas de chat o herramientas de implementación.

### Uso básico

Lo primero que querrá hacer con GitLab es crear un nuevo proyecto. Puede hacer esto haciendo clic en el icono "+" en la barra de herramientas. Se le pedirá el nombre del proyecto, a qué espacio de nombres debería pertenecer y cuál debería ser su nivel de visibilidad. La mayor parte de lo que especifica aquí no es permanente y se puede cambiar más adelante a través de la interfaz de configuración. Haga clic en "Crear proyecto" y listo.

Una vez que exista el proyecto, probablemente querrá conectarlo con un repositorio local de Git. Se puede acceder a cada proyecto a través de HTTPS o SSH, cualquiera de los cuales se puede utilizar para configurar un control remoto Git. Las URL están visibles en la parte superior de la página de inicio del proyecto. Para un repositorio local existente, este comando creará un control remoto con el nombre gitlab de la ubicación alojada:

```
$ git remote add gitlab https://server/namespace/project.git
```

Si no tiene una copia local del repositorio, simplemente puede hacer esto:

```
$ git clone https://server/namespace/project.git
```

La interfaz de usuario web proporciona acceso a varias vistas útiles del propio repositorio. La página de inicio de cada proyecto muestra la actividad reciente, y los enlaces en la parte superior lo llevarán a las vistas de los archivos del proyecto y al registro de confirmación.

### Trabajando juntos

La forma más sencilla de trabajar juntos en un proyecto de GitLab es otorgar a cada usuario acceso directo al repositorio de Git. Puede agregar un usuario a un proyecto yendo a la sección "Miembros" de la configuración de ese proyecto y asociando el nuevo usuario con un nivel de acceso (los diferentes niveles de acceso se discuten un poco en Grupos). Al otorgar a un usuario un nivel de acceso de "Desarrollador" o superior, ese usuario puede enviar confirmaciones y bifurcaciones directamente al repositorio.

Otra forma de colaboración más desacoplada es mediante el uso de solicitudes de combinación. Esta función permite a cualquier usuario que pueda ver un proyecto contribuir a él de forma controlada. Los usuarios con acceso directo pueden simplemente crear una rama, enviar confirmaciones a ella y abrir una solicitud de merge desde su rama hacia master o hacia cualquier otra rama. Los usuarios que no tienen permisos de inserción para un repositorio pueden "bifurcarlo" para crear su propia copia, enviar confirmaciones a su copia y abrir una solicitud de fusión desde su bifurcación al proyecto principal. Este modelo permite al propietario tener el control total de lo que ingresa al repositorio y cuándo, al mismo tiempo que permite las contribuciones de usuarios que no son de confianza.

Las solicitudes de merge y los problemas son las principales unidades de discusión de larga duración en GitLab. Cada solicitud de fusión permite una discusión línea por línea del cambio propuesto (que admite un tipo de revisión de código ligero), así como un hilo de discusión general general. Ambos pueden asignarse a usuarios u organizarse en hitos.

Esta sección se centra principalmente en las funciones de GitLab relacionadas con Git, pero como proyecto maduro, proporciona muchas otras funciones para ayudar a su equipo a trabajar en conjunto, como wikis de proyectos y herramientas de mantenimiento del sistema. Un beneficio de GitLab es que, una vez que el servidor está configurado y funcionando, rara vez necesitará modificar un archivo de configuración o acceder al servidor a través de SSH; la mayor parte de la administración y el uso general se pueden realizar a través de la interfaz del navegador.

### Opciones alojadas por terceros

Si no desea pasar por todo el trabajo involucrado en la configuración de su propio servidor Git, tiene varias opciones para alojar sus proyectos Git en un sitio de alojamiento externo dedicado. Si lo hace, ofrece una serie de ventajas: un sitio de alojamiento es generalmente rápido de configurar y fácil de iniciar proyectos, y no requiere mantenimiento ni supervisión del servidor. Incluso si configura y ejecuta su propio servidor internamente, es posible que aún desee utilizar un sitio de alojamiento público para su código fuente abierto; por lo general, es más fácil para la comunidad de código abierto encontrarlo y ayudarlo.

En estos días, tiene una gran cantidad de opciones de alojamiento para elegir, cada una con diferentes ventajas y desventajas. Para ver una lista actualizada, consulte la página de GitHosting en la wiki principal de Git en https://git.wiki.kernel.org/index.php/GitHosting .

Cubriremos el uso de GitHub en detalle, ya que es el host de Git más grande que existe y es posible que deba interactuar con proyectos alojados en él en cualquier caso, pero hay docenas más para elegir si no desea configurar su propio servidor Git.